In [68]:
using LinearAlgebra
using Plots
using Ripserer
using Distances
using Plots
using DataFrames, CSV
using DelimitedFiles
using AbstractAlgebra
using MultivariateStats #for pca
using IterativeSolvers
using NearestNeighbors
include("mv_function_bens_alg_no_save.jl")


mv (generic function with 1 method)

In [69]:
#Globals
K = 5 # in KNN
#OG_K = 10
ITER = 30    # iterations of algorithm
KKK = 2
OG_K = K
break_count = KKK #iterations before breakingxs
min_loss = 1000
min_loss_2 = 1000
input_file ="./cool_big_data.csv"
output_file ="./point_color"

a_ = []
b_ = []
c_ = []
d_ = []
e_ = []
f_ = []

loss_ = []
loss_2 = []


Any[]

In [70]:
# Loss function 0.5 instead of pi

function loss(points, coord_ab, coord_cd, coord_ef)
    a_cd = mod(coord_cd[points[1]],1.0)
    b_cd = mod(coord_cd[points[2]],1.0)

    c_ab = mod(coord_ab[points[3]],1.0)
    d_ab = mod(coord_ab[points[4]],1.0)

    e_ab = mod(coord_ab[points[5]],1.0)
    f_ab = mod(coord_ab[points[6]],1.0)

    e_cd = mod(coord_cd[points[5]],1.0)
    f_cd = mod(coord_cd[points[6]],1.0)

    a_ef = mod(coord_ef[points[1]],1.0)
    b_ef = mod(coord_ef[points[2]],1.0)
    c_ef = mod(coord_ef[points[3]],1.0)
    d_ef = mod(coord_ef[points[4]],1.0)


    dif_1 = 0.5 - mod(c_ab-d_ab,1.0)
    dif_2 = 0.5 - mod(e_ab-f_ab,1.0)
    dif_3 = 0.5 -  mod(a_cd-b_cd,1.0)
    dif_4 = 0.5 - mod(e_cd-f_cd,1.0)
    dif_5 = 0.5 -  mod(a_ef-b_ef,1.0)
    dif_6 = 0.5 -  mod(c_ef-d_ef,1.0)


    A = (0.25 - mod(c_ab - e_ab, 0.5))^2 + (0.25 - mod(e_ab - d_ab, 0.5))^2 #+ (0.25 - mod(d_ab - f_ab,0.5))^2 + + (0.25 - mod(f_ab - c_ab, 0.5))^2
    B =  (0.25 - mod(a_cd - e_cd, 0.5))^2 + (0.25 - mod(e_cd - b_cd, 0.5))^2 #+ (0.25 - mod(b_cd - f_cd,0.5))^2 + (0.25 - mod(f_cd - a_cd, 0.5))^2
    C =  (0.25 - mod(a_ef - c_ef, 0.5))^2 + (0.25 - mod(c_ef - b_ef, 0.5))^2  #+ (0.25 - mod(b_ef - d_ef,0.5))^2 + (0.25 - mod(d_ef - a_ef, 0.5))^2 

    #loss = max(A,B,C)
    loss = A + B + C
    loss +=  dif_1^2 + dif_2^2 +  dif_3^2 + dif_4^2 + dif_5^2 + dif_6^2  
    #loss += dif_1^2 + dif_2^2 + dif_3^2 + dif_4^2 + dif_5^2 + dif_6^2
    
    
    return loss
end

loss (generic function with 1 method)

In [71]:
# Load Data
matrix = readdlm(input_file, ',', Float64)
data = []
for row in 1:length(matrix[:,1])
    arr = [matrix[row,i] for i in 1:length(matrix[1,:])]
    push!(data, arr)
end

# Do zeroth iteration of algerithm with random points
l = length(matrix[:,1])
# Random distinct start points,
# Could be issues with distinct points yielding same piercing hole, and therefore cant do MV
#a_0,b_0,c_0,d_0,e_0,f_0 = rand(1:15), rand(15:30), rand(30:45), rand(45:60), rand(60:75), rand(75:100)
a_0,b_0,c_0 = rand(1:floor(Int, l/6)), rand(floor(Int, l/6):floor(Int, 2*l/6)), rand(floor(Int, 2*l/6):floor(Int,3*l/6))
d_0,e_0,f_0 = rand(floor(Int, 3*l/6):floor(Int,4*l/6)),rand(floor(Int, 4*l/6):floor(Int,4*l/6)),rand(floor(Int, 5*l/6):l)


# Store to see progress 
"""
push!(a_, a_0)
push!(b_, b_0)
push!(c_, c_0)
push!(d_, d_0)
push!(e_, e_0)
push!(f_, f_0)
"""

# Dinstance matrix for KNN
data_matrix = zeros(Float64, (3, length(data)))
    
for col in 1:length(data)
    data_matrix[:,col] = [data[col][1],data[col][2],data[col][3]]
end

kdtree = KDTree(data_matrix)





KDTree{StaticArraysCore.SVector{3, Float64}, Euclidean, Float64}
  Number of points: 80
  Dimensions: 3
  Metric: Euclidean(0.0)
  Reordered: true

In [72]:

#FIND BEST POINTS....

vec_0 = Array([1,0,0])
vec_1 = Array([0,1,0])
vec_2 = Array([0,0,1])

a_0_set, dists = knn(kdtree, vec_0,K, true)
c_0_set, dists = knn(kdtree, vec_1,K, true)
e_0_set, dists = knn(kdtree, vec_2,K, true)

vec_0 = Array([-1,0,0])
vec_1 = Array([0,-1,0])
vec_2 = Array([0,0,-1])

b_0_set, dists = knn(kdtree, vec_0,K, true)
d_0_set, dists = knn(kdtree, vec_1,K, true)
f_0_set, dists = knn(kdtree, vec_2,K, true)


push!(a_, a_0_set[1])
push!(b_, b_0_set[1])
push!(c_, c_0_set[1])
push!(d_, d_0_set[1])
push!(e_, e_0_set[1])
push!(f_, f_0_set[1])




1-element Vector{Any}:
 68

In [73]:


function update_points(points, index, x_0_set, min_loss, coord_ab, coord_cd, coord_ef, output_file)
    if index <=2
        coord_xy = coord_ab
        #nothing
    elseif index <=4
        output_file = output_file * "double"
        coord_xy = coord_cd
    else
        output_file = output_file * "tripple"
        coord_xy = coord_ef
    end
    x_0 = points[index]
    ok = false
    x_0_set = reverse(x_0_set)
    #min_loss = min_loss
    for i in 1:length(x_0_set)
        x_0_i = x_0_set[i]
        new_points = points
        new_points[index]  = x_0_i
        try
            # WE ONLY NEED TO COMPUTE ONE!!!! new coordinte
            coord_xy = mv(points = new_points, start_points = true, input_file = input_file, output_file = output_file, double_penetration = false, tripple  = false)
            if index <=2
                coord_ab = coord_xy
            elseif index <=4
                coord_cd = coord_xy
            else
                coord_ef = coord_xy
            end
            l = loss(new_points, coord_ab, coord_cd, coord_ef)
            if  l < min_loss
                min_loss = l
                x_0 = x_0_i
                ok = true
            end
        catch
            print("cant compute new coord")
            #nothing
        end
        if ok
            break
        end
    end
    
    return x_0, min_loss, coord_xy
end




update_points (generic function with 1 method)

In [74]:
#sisi =  [a_0_set[1]-1, b_0_set[1]-1, c_0_set[1]-1, d_0_set[1]-1, e_0_set[1]-1, f_0_set[1]-1]

In [75]:
  
for k in 1:ITER
    NN = length(a_)
    l = NN
    points = [a_[NN], b_[NN], c_[NN], d_[NN], e_[NN], f_[NN]]
    #print("pppp", points)
    try
        mv(points = points, start_points = true, input_file = input_file, output_file = output_file, double_penetration = true, tripple = true)
    catch
        println("Choose better starting points")
        l = length(a_)
        println("Init points ")
        println("[" * string(a_[l]-1) * "," * string(b_[l]-1) * "," * string(c_[l]-1) * "," 
        * string(d_[l]-1) * "," * string(e_[l]-1) *  "," * string(f_[l]-1) * "]")
        break

    end
    
    # indexing from 2 because c
    matrix = readdlm(output_file * ".csv", ',')
    coord_ab = matrix[2:end,4]

    matrix = readdlm(output_file * "double" * ".csv", ',')
    coord_cd = matrix[2:end,4]

    matrix = readdlm(output_file * "tripple" * ".csv", ',')
    coord_ef = matrix[2:end,4]



    
    a_0 = points[1]
    b_0 = points[2]
    c_0 = points[3]
    d_0 = points[4]
    e_0 = points[5]
    f_0 = points[6]
   


    if k == 1
        min_loss = loss([a_0,b_0,c_0,d_0, e_0, f_0], coord_ab, coord_cd, coord_ef)
        
        # Init Loss
        println("Init Loss: ", min_loss)
        push!(loss_, min_loss)

        l = length(a_)
        println("Init points ")
        println("[" * string(a_[l]-1) * "," * string(b_[l]-1) * "," * string(c_[l]-1) * "," 
        * string(d_[l]-1) * "," * string(e_[l]-1) *  "," * string(f_[l]-1) * "]")
    else
        println("Loss: ", min_loss)
        push!(loss_, min_loss)
    end
    

    ####
    #We move one both points in each pair
    ####

    # See if stationary
    old_b = b_0
    old_d = d_0
    old_f = f_0
    old_a = a_0
    old_c = c_0
    old_e = e_0

    old_loss = min_loss

    bol1 = true
    bol2 = true
    bol3 = true

    #vec_0 = Array([data[a_0][1],data[a_0][2],data[a_0][3]])
    vec_1 = Array([data[b_0][1],data[b_0][2],data[b_0][3]])
    vec_2 = Array([data[c_0][1],data[c_0][2],data[c_0][3]])
    vec_3 = Array([data[d_0][1],data[d_0][2],data[d_0][3]])
    vec_4 = Array([data[e_0][1],data[e_0][2],data[e_0][3]])
    vec_5 = Array([data[f_0][1],data[f_0][2],data[f_0][3]])
    
    #a_0_set, dists = knn(kdtree, vec_0,K, true)
    b_0_set, dists = knn(kdtree, vec_1,K, true)
    c_0_set, dists = knn(kdtree, vec_2,K, true)
    d_0_set, dists = knn(kdtree, vec_3,K, true)
    e_0_set, dists = knn(kdtree, vec_4,K, true)
    f_0_set, dists = knn(kdtree, vec_5,K, true)


    # find b_0^i and d_0^1 that minimizes loss



    a_0_i = a_0
    c_0_i = c_0
    e_0_i = e_0

    b_0_i = b_0
    d_0_i = d_0
    f_0_i = f_0
    
    #points, index, x_0_set, min_loss, coord_ab, coord_cd, coord_ef, output_file

    try
        b_0, min_loss,coord_ab = update_points([a_0,b_0,c_0,d_0, e_0, f_0], 2, b_0_set, min_loss,  coord_ab, coord_cd, coord_ef, output_file)
        println("Updated b_0 from ", old_b, " to ", b_0)
    catch
        println("Cant update b_0")
        b_0 = old_b
        min_loss = old_loss
    end
    old_loss = min_loss
    try
        c_0, min_loss, coord_cd = update_points([a_0,b_0,c_0,d_0, e_0, f_0],  3, c_0_set, min_loss,  coord_ab, coord_cd, coord_ef, output_file
        )
        println("Updated c_0 from ", old_c, " to ", c_0)
    catch
        println("Cant update c_0")
        c_0 = old_c
        min_loss = old_loss
    end
    old_loss = min_loss
    try
        d_0, min_loss, coord_cd = update_points([a_0,b_0,c_0,d_0, e_0, f_0],  4, d_0_set, min_loss,  coord_ab, coord_cd, coord_ef, output_file
        )
        println("Updated d_0 from ", old_d, " to ", d_0)
    catch
        println("Cant update d_0")
        d_0 = old_d
        min_loss = old_loss
    end
    old_loss = min_loss
    try
        e_0, min_loss, coord_ef = update_points([a_0,b_0,c_0,d_0, e_0, f_0], 5, e_0_set, min_loss,  coord_ab, coord_cd, coord_ef, output_file
        )
        println("Updated e_0 from ", old_e, " to ", e_0)
    catch
        println("Cant update e_0")
        e_0 = old_e
        min_loss = old_loss
    end
    old_loss = min_loss
    try
        f_0, min_loss, coord_ef = update_points([a_0,b_0,c_0,d_0, e_0,f_0],  6, f_0_set, min_loss,  coord_ab, coord_cd, coord_ef, output_file
        )
        println("Updated f_0 from ", old_f, " to ", f_0)
    catch
        println("Cant update f_0")
        f_0 = old_f
        min_loss = old_loss
    end
    old_loss = min_loss
    
 
    if old_b == b_0 && old_d == d_0 && old_f == f_0 && old_a == a_0 && old_c == c_0 && old_e == e_0
        println("FIXX POINTT")
        println("----------------------------------------------------------------")
        println("[" * string(a_0-1) * "," * string(b_0-1) * "," * string(c_0-1) * "," 
    * string(d_0-1) * "," * string(e_0-1) *  "," * string(f_0-1) * "]")
        println("----------------------------------------------------------------")

        break_count -=1
        K+=10
        if break_count == 0
            break
        end
 
    else
        K = OG_K
        break_count = KKK
    end
    push!(a_, a_0)
    push!(b_, b_0)
    push!(c_, c_0)
    push!(d_, d_0)
    push!(e_, e_0)
    push!(f_, f_0)


    l = length(a_)
    println("Done with: ", k, "   Break_count: ", break_count, " K = ", K)
    println("Points")
    println("[" * string(a_[l]-1) * "," * string(b_[l]-1) * "," * string(c_[l]-1) * "," 
    * string(d_[l]-1) * "," * string(e_[l]-1) *  "," * string(f_[l]-1) * "]")
    println("-------")

end

Init Loss: 0.03293877980542516
Init points 
[35,47,4,31,26,67]
Updated b_0 from 

48 to 48


Updated c_0 from 5 to 5
Updated d_0 from 

32 to 32


Updated e_0 from 27 to 27
Updated f_0 from 

...

Updated b_0 from 48 to 48
Updated c_0 from 

5 to 5


Updated d_0 from 32 to 32
Updated e_0 from 

27 to 27


Updated f_0 from 68 to 68
FIXX POINTT
----------------------------------------------------------------
[35,47,4,31,26,67]
----------------------------------------------------------------


In [76]:
loss_

2-element Vector{Any}:
 0.03293877980542516
 0.03293877980542516

In [77]:
println("Input:")
println("[" * string(a_[1]-1) * "," * string(b_[1]-1) * ","
 * string(c_[1]-1) * "," * string(d_[1]-1) *  "," * string(e_[1]-1) *  "," * string(f_[1]-1) * "]")

Input:
[35,47,4,31,26,67]


In [78]:
l = length(a_)
println("Output:")
println("[" * string(a_[l]-1) * "," * string(b_[l]-1) * "," * string(c_[l]-1) * "," 
* string(d_[l]-1) * "," * string(e_[l]-1) *  "," * string(f_[l]-1) * "]")

Output:
[35,47,4,31,26,67]
